#Part 1: Device Messages Analysis

Preview of Raw Device Messages
I am performing a basic select to verify that the data was imported correctly and to understand the column structure.

In [0]:
%sql
SELECT * FROM workspace.bronze.device_message_raw 
LIMIT 10;

###Peek at the data

Counting how many rows we have

In [0]:
%sql
SELECT 'device_message_raw' AS table_name, COUNT(*) AS rows FROM workspace.bronze.device_message_raw
UNION ALL
SELECT 'Rapid_step_test_raw', COUNT(*) FROM workspace.bronze.Rapid_step_test_raw;

###Basic Summary

Counting message per device

In [0]:
%sql
SELECT device_id, COUNT(*) AS messages FROM workspace.bronze.device_message_raw GROUP BY 1 ORDER BY 2 DESC LIMIT 10;

###Identifying Unique Devices
This query counts how many unique devices are sending messages, which helps identify the scope of the dataset.

In [0]:
%sql
SELECT count(distinct device_id) as unique_devices FROM workspace.bronze.device_message_raw;

###Quality Check: Missing Data
Checking for null values in the 'message' or 'timestamp' columns to identify records that might need cleaning in the Silver layer.

In [0]:
%sql
SELECT * FROM workspace.bronze.device_message_raw 
WHERE device_id IS NULL OR timestamp IS NULL;

#Part 2: Rapid Step Test Analysis

Preview of Rapid Step Test Data
Inspecting the step test results to see how user performance data is recorded.

In [0]:
%sql
SELECT * FROM workspace.bronze.Rapid_step_test_raw LIMIT 10;

###Finding High-Performance Results
Filtering the data to see tests where the 'step_count' or 'score' is above average, which helps define "successful" test runs.

In [0]:
%sql
SELECT * FROM workspace.bronze.Rapid_step_test_raw 
ORDER BY total_steps DESC 
LIMIT 5;

###Identifying Top Performers
I am querying the data to find the records with the highest 'total_steps'. This helps identify the upper range of the physical activity data collected.

In [0]:
%sql
SELECT customer, total_steps, step_points 
FROM workspace.bronze.Rapid_step_test_raw 
ORDER BY total_steps DESC 
LIMIT 10;

###Data Quality: Duration Validation
Checking for potential data errors where the 'stop_time' might be earlier than the 'start_time', which would indicate a system glitch.

In [0]:
%sql
SELECT customer, start_time, stop_time 
FROM workspace.bronze.Rapid_step_test_raw 
WHERE stop_time < start_time;

###Clean and cast distance

Created a cleaned view for convenience during lab


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT  
  'device_message_raw' AS source_table,
  device_id,  
  TRY_CAST(REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '') AS DOUBLE) AS distance_cm,  
  CAST(timestamp AS BIGINT) AS ts_ms,
  message_origin AS messageOrigin,
  sensor_type,  
  message
FROM workspace.bronze.device_message_raw;

###Explode stepPoints (array → rows)

Handling the array explosion for stepPoints.

In [0]:
%sql
WITH exploded AS (
  SELECT customer, device_id, start_time, 
         posexplode(step_points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_test_raw
)
SELECT customer, device_id, COUNT(*) AS steps, AVG(step_ms) AS avg_step_ms
FROM exploded GROUP BY 1, 2 ORDER BY steps DESC LIMIT 10;

###Time-window join

This query shows how many sensor readings occurred during each test. We will reuse these aggregates as features for ML later.

In [0]:
%sql
SELECT t.customer, t.device_id, t.start_time, t.stop_time,
       COUNT(m.ts_ms) AS readings_in_window,
       AVG(m.distance_cm) AS avg_cm_in_window
FROM workspace.bronze.rapid_step_test_raw t
JOIN device_messages_clean m ON m.device_id = t.device_id 
  AND m.ts_ms BETWEEN t.start_time AND t.stop_time
GROUP BY 1, 2, 3, 4 ORDER BY readings_in_window DESC LIMIT 20;